Prediction `Number of Sunspots` with several models,**LieanrRegression, DecisionTree, RandomForest, Ridge**. Finally I got the best `MAE` almost **1.6**.

In [ ]:
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import matplotlib as mlp
import seaborn as sns
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import OneHotEncoder, StandardScaler # OneHE is used for convert 'str' data to numerical
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline  # multi preproseccing
from sklearn.base import BaseEstimator, TransformerMixin  #use to construct eg. AttributesAdder 
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

In [ ]:
df=pd.read_csv('/kaggle/input/daily-sun-spot-data-1818-to-2019/sunspot_data.csv')
df.head(10)

In [ ]:
df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
cor_mx=df.corr()
cor_mx['Number of Sunspots'].sort_values(ascending=False)

In [ ]:
df_2018=df[df['Year']==2018]

In [ ]:
plt.figure(figsize=(50,30))
sns.set(font_scale = 2.5)
sns.barplot(data=df_2018, y='Month', x='Number of Sunspots', hue='Day', orient='h')
plt.title('Number of sunspot in each day of Jan 2018')

In [ ]:
df_2018.head()

In [ ]:
plt.figure(figsize=(8,6))
sns.set(font_scale=0.8)
sns.barplot(data=df_2018, y=df_2018['Day'][df_2018['Month']==1], x='Number of Sunspots', orient='h', color='blue')
plt.title('Number of sunspot in each day of Jan 2018')

In [ ]:
plt.figure(figsize=(8,6))
sns.set(font_scale=0.8)
sns.barplot(data=df_2018, y=df_2018['Day'][df_2018['Month']==2], x='Number of Sunspots', orient='h', color='red')
plt.title('Number of sunspot in each day of Feb 2018')

In [ ]:
plt.figure(figsize=(8,6))
sns.set(font_scale=0.8)
sns.barplot(data=df_2018, y=df_2018['Day'][df_2018['Month']==6], x='Number of Sunspots', orient='h', color='green')
plt.title('Number of sunspot in each day of Jun 2018')

In [ ]:
df.isnull().sum()

In [ ]:
df.drop(['Month','Day','Observations', 'Indicator'], axis=1, inplace=True)

In [ ]:
df

In [ ]:
from sklearn.model_selection import train_test_split

train, test=train_test_split(df, test_size=0.2, random_state=42)
train

In [ ]:
X=train.drop('Number of Sunspots', axis=1, inplace=False)
y=train['Number of Sunspots'].copy()

In [ ]:
scale=StandardScaler()
X_scaled=scale.fit_transform(X)

In [ ]:
lin_reg=LinearRegression()

lin_reg.fit(X_scaled, y)

In [ ]:
lin_scores=cross_val_score(lin_reg, X_scaled, y, scoring='neg_mean_absolute_error', cv=10)
lin_mae=-lin_scores
lin_mae

def cost_fun(scores):
    print('Scores:', scores,
         'Mean:', scores.mean(),
         'Standard Deviation:', scores.std())

In [ ]:
cost_fun(lin_mae)

In [ ]:
from sklearn.linear_model import Ridge, Lasso

ridge=Ridge()
ridge.fit(X_scaled, y)
ridge_scores=cross_val_score(ridge, X_scaled, y, cv=10, scoring='neg_mean_absolute_error')
ridge_mae=-ridge_scores

cost_fun(ridge_mae)

In [ ]:
param_grid=({
    'alpha': np.logspace(-5,5,10)
})

ridge_search=GridSearchCV(ridge, param_grid, cv=10, 
                         scoring='neg_mean_absolute_error',
                         return_train_score=True)
ridge_search.fit(X_scaled,y)


In [ ]:
ridge_search.best_params_

In [ ]:
ridge_model=ridge_search.best_estimator_
ridge_model.fit(X_scaled,y)
ridge_model_scores=cross_val_score(ridge_model, X_scaled, y, cv=10, scoring='neg_mean_absolute_error')
ridge_model_mae=-ridge_model_scores

cost_fun(ridge_model_mae)

In [ ]:
tree_reg=DecisionTreeRegressor()
tree_reg.fit(X_scaled, y)
tree_scores=cross_val_score(tree_reg, X_scaled, y, cv=10, scoring='neg_mean_absolute_error')
tree_mae=-tree_scores

cost_fun(tree_mae)

In [ ]:
random_forest=RandomForestRegressor()
random_forest.fit(X_scaled, y)

forest_score=cross_val_score(random_forest, X_scaled, y, cv=10, scoring='neg_mean_absolute_error')
forest_mae=-forest_score

cost_fun(forest_mae)

### Training Model with polynomial features

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

poly=PolynomialFeatures(degree=3)
X_poly=poly.fit_transform(X_scaled)

lin_reg.fit(X_poly, y)

In [ ]:
lin_scores=cross_val_score(lin_reg, X_poly, y, scoring='neg_mean_absolute_error', cv=10)
lin_mae=-lin_scores

cost_fun(lin_mae)

In [ ]:
tree_reg=DecisionTreeRegressor()
tree_reg.fit(X_poly, y)

tree_scores=cross_val_score(tree_reg, X_poly, y, cv=10, scoring='neg_mean_absolute_error')
tree_mae=-tree_scores

cost_fun(tree_mae)

In [ ]:
random_forest=RandomForestRegressor()
random_forest.fit(X_poly, y)

forest_score=cross_val_score(random_forest, X_poly, y, cv=5, scoring='neg_mean_absolute_error')
forest_mae=-forest_score

cost_fun(forest_mae)

It ( Polynomial) does not make sense

In [ ]:

param_grid = [
    # Try 20 (4*5) combination of hyperparameters
    {'n_estimators': [10, 20, 40], 'max_features': [2, 3]},
    # Try bootstrap as False with 12 combination
    {'bootstrap': [False], 'n_estimators': [20, 40], 'max_features': [2, 3]},
]
forest_reg = RandomForestRegressor(random_state=42)
# Train across 5 fold, that is a total of (16 + 12)*5 rounds of training
forest_search = GridSearchCV(forest_reg, param_grid, cv=5,
                           scoring="neg_mean_absolute_error",
                           return_train_score=True)
forest_search.fit(X_scaled, y)

In [ ]:
forest_search.best_params_

In [ ]:
forest_model=forest_search.best_estimator_

forest_model.fit(X_scaled, y)

forest_model_score=cross_val_score(forest_model, X_scaled, y, cv=5, scoring='neg_mean_absolute_error')
forest_model_rmse=np.sqrt(-forest_model_score)

cost_fun(forest_model_rmse)

let's go through our `test_set` for final `MAE`

In [ ]:
test

In [ ]:
X_test=test.drop('Number of Sunspots', axis=1, inplace=False)
y_test=test['Number of Sunspots'].copy()

X_test_scaled=scale.transform(X_test)
final_prediction=forest_model.predict(X_test_scaled)
final_mae=mean_absolute_error(final_prediction, y_test)
final_mae=np.sqrt(final_mae)
final_mae